In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import random

labels = ['rickshaw', 'easybike', 'cng']
IMG_SIZE = 50

def get_data_train(data_dir):
  data = []
  for label in labels:
    path = os.path.join(data_dir, label)
    print(path)
    class_num = labels.index(label)
    print(class_num)
    for img in os.listdir(path):
      try:
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        data.append([new_array, class_num])
      except Exception as e:
        print(e)
  return np.array(data)

In [3]:
dataset = get_data_train("./data/")

./data/rickshaw
0
./data/easybike
1
./data/cng
2


c:\users\it park\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [4]:
X = []
y = []

for feature, label in dataset:
  X.append(feature)
  y.append(label)

In [5]:
from sklearn.preprocessing import LabelBinarizer # LabelBinarizer
label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(y)

In [6]:
from sklearn.model_selection import train_test_split
seed = 42
test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [7]:
X_train = np.array(X_train) / 255
X_test = np.array(X_test) / 255

In [8]:
X_train = X_train.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y_train = np.array(y_train)

X_test = X_test.reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y_test = np.array(y_test)

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Conv2D, Activation, Dropout, Dense, MaxPooling2D
from tensorflow.keras.models import Sequential

In [10]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same", input_shape=X_train.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.1))

# model.add(Conv2D(256, (3, 3), padding="same"))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(2, 2))
# model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(128, activation="relu"))

model.add(Dense(len(labels)))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), shuffle=True)

Epoch 1/100
5/5 [==============================] - 1s 228ms/step - loss: 1.5565 - accuracy: 0.3881 - val_loss: 1.0961 - val_accuracy: 0.3235
Epoch 2/100
5/5 [==============================] - 1s 140ms/step - loss: 1.1036 - accuracy: 0.3881 - val_loss: 1.0958 - val_accuracy: 0.3824
Epoch 3/100
5/5 [==============================] - 1s 144ms/step - loss: 1.0836 - accuracy: 0.4254 - val_loss: 1.0895 - val_accuracy: 0.3824
Epoch 4/100
5/5 [==============================] - 1s 115ms/step - loss: 1.0787 - accuracy: 0.4254 - val_loss: 1.0805 - val_accuracy: 0.3824
Epoch 5/100
5/5 [==============================] - 1s 108ms/step - loss: 1.0617 - accuracy: 0.4328 - val_loss: 1.0524 - val_accuracy: 0.3824
Epoch 6/100
5/5 [==============================] - 1s 125ms/step - loss: 1.0409 - accuracy: 0.4328 - val_loss: 1.0061 - val_accuracy: 0.7647
Epoch 7/100
5/5 [==============================] - 1s 144ms/step - loss: 1.0355 - accuracy: 0.5000 - val_loss: 0.9436 - val_accuracy: 0.7941
Epoch 8/100
5

Epoch 59/100
5/5 [==============================] - 1s 132ms/step - loss: 0.0094 - accuracy: 1.0000 - val_loss: 0.2737 - val_accuracy: 0.9412
Epoch 60/100
5/5 [==============================] - 1s 132ms/step - loss: 0.0084 - accuracy: 1.0000 - val_loss: 0.3712 - val_accuracy: 0.8824
Epoch 61/100
5/5 [==============================] - 1s 128ms/step - loss: 0.0117 - accuracy: 1.0000 - val_loss: 0.3074 - val_accuracy: 0.9118
Epoch 62/100
5/5 [==============================] - 1s 133ms/step - loss: 0.0095 - accuracy: 1.0000 - val_loss: 0.3264 - val_accuracy: 0.9118
Epoch 63/100
5/5 [==============================] - 1s 104ms/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.4123 - val_accuracy: 0.8824
Epoch 64/100
5/5 [==============================] - 1s 110ms/step - loss: 0.0102 - accuracy: 1.0000 - val_loss: 0.3942 - val_accuracy: 0.9118
Epoch 65/100
5/5 [==============================] - 1s 114ms/step - loss: 0.0084 - accuracy: 1.0000 - val_loss: 0.3684 - val_accuracy: 0.9118
Epoch 

In [11]:
pred = np.argmax(model.predict(X_test), axis=-1)
pred

array([2, 1, 2, 0, 2, 2, 2, 0, 2, 1, 0, 2, 2, 0, 1, 1, 1, 2, 2, 2, 0, 0,
       0, 0, 0, 0, 1, 2, 1, 0, 0, 0, 2, 2], dtype=int64)

In [12]:
model.save("cnn.model")
model.save("c.h5")
labels = ['rickshaw', 'easybike', 'cng']
def prepare(filepath):
    img_array = cv2.imread(filepath, cv2.IMREAD_COLOR)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

model = tf.keras.models.load_model("c.h5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: cnn.model\assets


In [13]:
prediction = model.predict_classes([prepare("OBHAI-CNG-2.jpg")])
print(labels[(prediction[0])])

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
cng


In [14]:
from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("a.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("a.h5")
print("Saved model to disk")

Using TensorFlow backend.


Saved model to disk
